In [1]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow as tf

object_detection_path = '../Github/models/research/object_detection'

from PIL import Image
import sys
sys.path.insert(0, object_detection_path)
from utils import dataset_util
from collections import namedtuple, OrderedDict

modes=[
    ['./csvs/train_labels.csv',object_detection_path+'/data/train.record'],
    ['./csvs/test_labels.csv', object_detection_path+'/data/test.record']
]
predefined_classes_file = 'predefined_classes.txt'
label_map_file = object_detection_path+'/data/object_detection.pbtxt'

predefined_classes =[]

with open(predefined_classes_file, 'r') as myfile:
    predefined_classes=myfile.read().split()

In [2]:
def create_label_map(classes):
    content = ''
    count = 1
    for clss in classes:
        content += 'item {\n'+'id:'+str(count)+'\n'+'name:\''+clss+'\'\n'+'}\n'
        count+=1
    with open(label_map_file, "w") as text_file:
        print(content, file=text_file)
        


In [3]:
def class_text_to_int(row_label):
    return predefined_classes.index(row_label)+1

def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def run():
    create_label_map(predefined_classes)
    for mode in modes:
        writer = tf.python_io.TFRecordWriter(mode[1])
        path = os.path.join(os.getcwd(), 'images')
        examples = pd.read_csv(mode[0])
        grouped = split(examples, 'filename')
        for group in grouped:
            tf_example = create_tf_example(group, path)
            writer.write(tf_example.SerializeToString())

        writer.close()
        output_path = os.path.join(os.getcwd(), mode[1])
        print('Successfully created the TFRecords: {}'.format(output_path))

run()

Successfully created the TFRecords: /usr/src/object_detection/workspace/../Github/models/research/object_detection/data/train.record
Successfully created the TFRecords: /usr/src/object_detection/workspace/../Github/models/research/object_detection/data/test.record
